# PageRank algorithm coded without any library based on a set of html pages in a sub-folder.

In [ ]:
import re

#Construct webpages graph from pages in a directory
webpages = {}
for element in os.listdir('pages'): #Loop on all elements of toyset's folder
    webpages[element] = len(webpages) + 1

graph = open("./graph.txt", "w") #Create graph.txt file
for wb in webpages:
    f=open("pages/" + wb, "r", encoding="utf8")
    if f.mode == 'r':
        content = f.read()
        datas = re.findall(r'a href="[\S]*.html[\S]', content) #find all links on the webpage
        for e in datas:
            graph.write(str(webpages[wb]) + " " + str(webpages[e.split('"')[1]]) + "\r") #write in graph.txt the edge between the two pages
    f.close()
graph.close()        

In [ ]:
#create set of functions to deal with matrices
def matricial_product(M,PI):
    result = [0.0 for i in range(len(PI))]
    for e in M:
        i, j, value = e.split(' ')
        i = int(i) - 1
        j = int(j) - 1
        result[int(i)] += float(value) * float(PI[int(j)])
    return result

def scalar_matrix_product(scalar, M):  
    result = []  
    for e in M:
        args = e.split(' ')
        args[2] = str(float(args[2]) * scalar)
        result.append(' '.join(args))
    return result
    
def get_norm1(A):
    sum = 0
    for e in A:
        sum += abs(e)
    return sum

In [ ]:
def PageRankAlgorithm(graph_path, beta, epsilon):
    G = []
    with open(graph_path, 'r')  as graph_file:
        for line in graph_file.readlines(): #Open graph file, read the lines and creates the graph in a frame of [(i,j),...]
            args = line.split(' ')
            G.append((int(args[0]), int(args[1])))

    #Compute Mg matrix
    #Start create a dictionnary in which the key is a node and the value is a list of the page with which there are one or more links from the key node.
    graph_dictionnary = {}
    for e in G:
        if e[0] not in graph_dictionnary:
            graph_dictionnary[e[0]] = []
        if e[1] not in graph_dictionnary[e[0]] and e[1] != e[0]:
            graph_dictionnary[e[0]].append(e[1])

    n = len(graph_dictionnary.keys())

    #As there is no dead-ends we can suppose that there is at least an edge starting from each point (key value)
    M = []
    for i in range(n):
        keys_i = [*graph_dictionnary][i]
        for j in range(n):
            keys_j = [*graph_dictionnary][j]
            if (keys_i == keys_j):
                continue
            if keys_i in graph_dictionnary[keys_j]:
                M.append(" ".join([str(keys_i), str(keys_j), str(1/len(graph_dictionnary[keys_j]))]))

    #Initialize pi vector
    pi = []
    pi.append([1/n for i in range(n)])
    iteration = 0
    while 1:
        sparse_product = [beta * e + (1-beta)/n for e in matricial_product(M, pi[-1])]
        pi.append(sparse_product)
        if get_norm1([pi[-1][i] - pi[-2][i] for i in range(len(pi[-1]))]) < epsilon:
            break
        iteration += 1
        if iteration >= 100:
            break

    return pi[-1]

## How to use algorithm : 

In [ ]:
vector = PageRankAlgorithm2("./graph.txt", 1, 0.1)
max, min = 0, 1
max_index, min_index = 0,0
for i in range(len(vector)):
    if vector[i] > max:
        max = vector[i]
        max_index = i
    if vector[i] < min:
        min = vector[i]
        min_index = i
    print(str(vector[i]) + " --> " + [*webpages][i])

print()
print("Max value : " + [*webpages][max_index] + " (" + str(max) + ")")
print("Min value : " + [*webpages][min_index] + " (" + str(min) + ")")

## A function if you want to delete dead-ends

In [ ]:
def remove_dead_ends(graph_path):
    G = []
    with open(graph_path, 'r')  as graph_file: #Load graph datas from txt files
        for line in graph_file.readlines(): #Read all lines and recreate the graph in a frame of [(i,j),...]
            args = line.split(' ')
            G.append((int(args[0]), int(args[1])))

    dead_ends = [-1] #Initialize a virtual dead-end to enter in the loop.
    while len(dead_ends) > 0: #While we found dead-end at previous operation continue to remove dead-ends
        start = [] #start will represent all the nodes that are the start of at least one arrow
        destination = [] #destination will represent all the nodes that are the end of at least one arrow
        for i, j in G:
            if j not in destination:
                destination.append(j)
            if i not in start:
                start.append(i)

        dead_ends = [] #re-instanciate dead-end list, 0 dead-end found yet
        for d in destination:
            if d not in start: #if a node is a destination and not a start --> that's a dead-end
                dead_ends.append(d)
        
        if len(dead_ends) == 0: #if there is no dead-end, the job is done !
            break
        
        new_g = [] #else recompute the new graph without the dead-end point <=> without the arrows that go to the dead end points
        for i, j in G:
            if j in dead_ends:
                continue
            new_g.append((i,j))
        G = new_g
    return G